In [ ]:
# Removes lint errors from VS Code
from typing import Dict, TYPE_CHECKING, Tuple, List
from functools import reduce

if TYPE_CHECKING:
    import kedro

    catalog: kedro.io.data_catalog.DataCatalog
    session: kedro.framework.session.session.KedroSession
    catalog: kedro.io.data_catalog.DataCatalog
    pipelines: Dict[str, kedro.pipeline.pipeline.Pipeline]

import numpy as np
import pandas as pd


In [ ]:
from pasteur.transform import TableTransformer

table: pd.DataFrame = catalog.load("mimic_tab_admissions.wrk.bhr_table")
trn: TableTransformer = catalog.load("mimic_tab_admissions.wrk.trn_table")

In [ ]:
attr = trn.get_attributes("bhr", table)

In [ ]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
def calc_marginal(
    data: np.ndarray,
    domain: np.ndarray,
    x: list[int],
    p: list[int],
    rm_zeros: bool = False,
):
    """Calculates the 1 way and 2 way marginals between the set of columns in x
    and the set of columns in p."""

    sub_data = data[:, x + p]
    sub_domain = domain[x + p]

    margin, _ = np.histogramdd(sub_data, sub_domain)
    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    x_idx = tuple(range(len(x)))
    p_idx = tuple(range(-len(p), 0))

    x_mar = np.sum(margin, axis=p_idx).reshape(-1)
    p_mar = np.sum(margin, axis=x_idx).reshape(-1)
    j_mar = margin.reshape((len(x_mar), len(p_mar)))

    return j_mar, x_mar, p_mar

In [ ]:
# synth = PrivBayesSynth()
# synth.bake({"table": attr}, {"table": table}, {})

data = table.to_numpy(dtype="uint16")
domain = data.max(axis=0) + 1

# %lprun -f calc_marginal calc_marginal(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15])
j_mar1, x_mar1, p_mar1 = calc_marginal(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15])

In [ ]:
def calc_marginal_2(
    data: np.ndarray,
    domain: np.ndarray,
    x: list[int],
    p: list[int],
    rm_zeros: bool = False,
):
    """Calculates the 1 way and 2 way marginals between the set of columns in x
    and the set of columns in p."""

    sub_data = data[:, x + p]
    sub_domain = domain[x + p]

    x_dom = reduce(lambda a, b: a*b, domain[x], 1)
    p_dom = reduce(lambda a, b: a*b, domain[p], 1)
    idx = np.zeros((len(sub_data)), dtype="int64")
    for col, dom in zip(sub_data.transpose(), sub_domain):
        idx = idx*dom + col
    
    counts = np.bincount(idx, minlength=x_dom*p_dom)
    margin = counts.reshape(x_dom, p_dom).astype("float")

    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    j_mar = margin
    x_mar = np.sum(margin, axis=1)
    p_mar = np.sum(margin, axis=0)

    return j_mar, x_mar, p_mar
    
j_mar2, x_mar2, p_mar2 = calc_marginal_2(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15])

In [ ]:
assert np.all(j_mar1 == j_mar2)
assert np.all(x_mar1 == x_mar2)
assert np.all(p_mar1 == p_mar2)

In [ ]:
%timeit calc_marginal(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_760504/2048533878.py:1 in <cell line: 1>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_760504/2048533878.py'                       │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:241 │
│ 8 in run_line_magic                                                                              │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/decorator.py:232 in fun              │
│                                                                                                  │
│   229 │   │   def fun(*args, **kw):                                                              │
│   230 │   │   │   if not kwsyntax:                                                               │
│   231 │   │   │   │   args, kw = fix(args, kw, sig)                                              │
│ ❱ 232 │   │   │   return caller(func, *(extras + args), **kw)                                    │
│   233 │   fun.__name__ = func.__name__                                                           │
│   234 │   fun.__doc__ = func.__doc__                                                             │
│   235 │   fun.__wrapped__ = func                                                                 │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magic.py:187 in         │
│ <lambda>                                                                                         │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)                                               │
│   188 │   │                                                                                      │
│   189 │   │   if callable(arg):                                                                  │
│   190 │   │   │   # "Naked" decorator call (just @foo, no args)                                  │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magics/execution.py:118 │
│ 0 in timeit                                                                                      │
│                                                            

In [ ]:
%timeit calc_marginal_2(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

9.53 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f calc_marginal_2 calc_marginal_2(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

Timer unit: 1e-06 s

Total time: 0.008019 s
File: /tmp/ipykernel_760504/4051514708.py
Function: calc_marginal_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calc_marginal_2(
     2                                               data: np.ndarray,
     3                                               domain: np.ndarray,
     4                                               x: list[int],
     5                                               p: list[int],
     6                                               rm_zeros: bool = False,
     7                                           ):
     8                                               """Calculates the 1 way and 2 way marginals between the set of columns in x
     9                                               and the set of columns in p."""
    10                                           
    11         1        902.0    902.0     11.2      sub_data = data[:, x

In [ ]:
import numexpr as ne

def calc_marginal_3(
    data: np.ndarray,
    domain: np.ndarray,
    x: list[int],
    p: list[int],
    rm_zeros: bool = False,
):
    """Calculates the 1 way and 2 way marginals between the set of columns in x
    and the set of columns in p."""

    xp = x + p
    x_dom = reduce(lambda a, b: a*b, domain[x], 1)
    p_dom = reduce(lambda a, b: a*b, domain[p], 1)

    idx = np.zeros((len(data)), dtype="int64")
    for col in xp:
        idx = domain[col]*idx + data[:, col]
    
    counts = np.bincount(idx, minlength=x_dom*p_dom)
    margin = counts.reshape(x_dom, p_dom).astype("float")

    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    j_mar = margin
    x_mar = np.sum(margin, axis=1)
    p_mar = np.sum(margin, axis=0)

    return j_mar, x_mar, p_mar
    
j_mar3, x_mar3, p_mar3 = calc_marginal_3(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15])

In [ ]:
%timeit calc_marginal_3(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

6.71 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f calc_marginal_3 calc_marginal_3(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

Timer unit: 1e-06 s

Total time: 0.007592 s
File: /tmp/ipykernel_760504/3634434575.py
Function: calc_marginal_3 at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def calc_marginal_3(
     4                                               data: np.ndarray,
     5                                               domain: np.ndarray,
     6                                               x: list[int],
     7                                               p: list[int],
     8                                               rm_zeros: bool = False,
     9                                           ):
    10                                               """Calculates the 1 way and 2 way marginals between the set of columns in x
    11                                               and the set of columns in p."""
    12                                           
    13         1          2.0      2.0      0.0      xp = x + p
    14   

In [ ]:
assert np.all(j_mar1 == j_mar3)
assert np.all(x_mar1 == x_mar3)
assert np.all(p_mar1 == p_mar3)

In [ ]:
def calc_marginal_4(
    data: np.ndarray,
    domain: np.ndarray,
    x: list[int],
    p: list[int],
    rm_zeros: bool = False,
):
    """Calculates the 1 way and 2 way marginals between the set of columns in x
    and the set of columns in p."""

    xp = x + p
    x_dom = reduce(lambda a, b: a*b, domain[x], 1)
    p_dom = reduce(lambda a, b: a*b, domain[p], 1)

    idx = np.zeros((len(data)), dtype="int64")
    mul = 1
    for col in reversed(xp):
        # idx += mul*data[:, col]
        np.add(idx, mul*data[:, col], out=idx)
        mul *= domain[col]
    
    counts = np.bincount(idx, minlength=x_dom*p_dom)
    margin = counts.reshape(x_dom, p_dom).astype("float")

    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    j_mar = margin
    x_mar = np.sum(margin, axis=1)
    p_mar = np.sum(margin, axis=0)

    return j_mar, x_mar, p_mar
    
j_mar4, x_mar4, p_mar4 = calc_marginal_4(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15])

In [ ]:
%timeit calc_marginal_4(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

4.89 ms ± 23.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f calc_marginal_4 calc_marginal_4(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

Timer unit: 1e-06 s

Total time: 0.005484 s
File: /tmp/ipykernel_760504/1461518459.py
Function: calc_marginal_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calc_marginal_4(
     2                                               data: np.ndarray,
     3                                               domain: np.ndarray,
     4                                               x: list[int],
     5                                               p: list[int],
     6                                               rm_zeros: bool = False,
     7                                           ):
     8                                               """Calculates the 1 way and 2 way marginals between the set of columns in x
     9                                               and the set of columns in p."""
    10                                           
    11         1          2.0      2.0      0.0      xp = x + p
    12   

In [ ]:
assert np.all(j_mar1 == j_mar4)
assert np.all(x_mar1 == x_mar4)
assert np.all(p_mar1 == p_mar4)

In [ ]:
def calc_marginal_5(
    data: np.ndarray,
    domain: np.ndarray,
    x: list[int],
    p: list[int],
    rm_zeros: bool = False,
):
    """Calculates the 1 way and 2 way marginals between the set of columns in x
    and the set of columns in p."""

    xp = x + p
    x_dom = reduce(lambda a, b: a*b, domain[x], 1)
    p_dom = reduce(lambda a, b: a*b, domain[p], 1)

    idx = np.zeros((len(data)), dtype="uint16")
    tmp = np.empty((len(data)), dtype="uint16")
    mul = 1
    for col in reversed(xp):
        # idx += mul*data[:, col]
        np.add(idx, np.multiply(mul,data[:, col],out=tmp), out=idx)
        mul *= domain[col]
    
    counts = np.bincount(idx, minlength=x_dom*p_dom)
    margin = counts.reshape(x_dom, p_dom).astype("float")

    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    j_mar = margin
    x_mar = np.sum(margin, axis=1)
    p_mar = np.sum(margin, axis=0)

    return j_mar, x_mar, p_mar
    
j_mar5, x_mar5, p_mar5 = calc_marginal_5(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15])

In [ ]:
%timeit calc_marginal_5(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

3.7 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f calc_marginal_4 calc_marginal_4(data, domain, [0,1,2,3,4], [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

Timer unit: 1e-06 s

Total time: 0.005653 s
File: /tmp/ipykernel_760504/1461518459.py
Function: calc_marginal_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calc_marginal_4(
     2                                               data: np.ndarray,
     3                                               domain: np.ndarray,
     4                                               x: list[int],
     5                                               p: list[int],
     6                                               rm_zeros: bool = False,
     7                                           ):
     8                                               """Calculates the 1 way and 2 way marginals between the set of columns in x
     9                                               and the set of columns in p."""
    10                                           
    11         1          2.0      2.0      0.0      xp = x + p
    12   

In [ ]:
assert np.all(j_mar1 == j_mar5)
assert np.all(x_mar1 == x_mar5)
assert np.all(p_mar1 == p_mar5)

In [ ]:
def calc_marginal_1way(
    data: np.ndarray, domain: np.ndarray, x: list[int], rm_zeros: bool = False
):
    """Calculates the 1 way marginal of x, returned as a 1D array."""

    x_dom = reduce(lambda a, b: a * b, domain[x], 1)

    idx = np.zeros((len(data)), dtype="int64")
    mul = 1
    for col in reversed(x):
        # idx += mul*data[:, col]
        np.add(idx, mul * data[:, col], out=idx)
        mul *= domain[col]

    counts = np.bincount(idx, minlength=x_dom)
    margin = counts.astype("float")
    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    return margin

mar = calc_marginal_1way(data, domain, [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

In [ ]:
%timeit calc_marginal_1way(data, domain, [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

2.61 ms ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f calc_marginal_1way (data, domain, [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

Timer unit: 1e-06 s

Total time: 0 s
File: /tmp/ipykernel_760504/268377759.py
Function: calc_marginal_1way at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def calc_marginal_1way(
     2                                               data: np.ndarray, domain: np.ndarray, x: list[int], rm_zeros: bool = False
     3                                           ):
     4                                               """Calculates the 1 way marginal of x, returned as a 1D array."""
     5                                           
     6                                               x_dom = reduce(lambda a, b: a * b, domain[x], 1)
     7                                           
     8                                               idx = np.zeros((len(data)), dtype="int64")
     9                                               mul = 1
    10                                               for col in reversed(x):
    11     

In [ ]:
def calc_marginal_1way_old(
    data: np.ndarray, domain: np.ndarray, x: list[int], rm_zeros: bool = False
):
    """Calculates the 1 way marginal of x, returned as a 1D array."""
    
    sub_data = data[:, x]
    sub_domain = domain[x]
    margin, _ = np.histogramdd(sub_data, sub_domain)
    margin /= margin.sum()
    if rm_zeros:
        # Mutual info turns into NaN without this
        margin += 1e-24

    return margin.reshape(-1)

mar2 = calc_marginal_1way_old(data, domain, [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

In [ ]:
%timeit calc_marginal_1way_old(data, domain, [5,6,7,8,9,10,11,12,13,14,15,16,17,18])

516 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
assert np.all(mar == mar2)

In [ ]:
model = catalog.load("mimic_tab_admissions.privbayes.model")

In [ ]:
model.fit({"table": table}, {})

[08/08/22 11:34:55] INFO     Deterministic check: random number after     PrivBayesSynth.fit():          ]8;id=637358;file:///mnt/ext/projects/pasteur/src/pasteur/synth/base.py\base.py]8;;\:]8;id=191348;file:///mnt/ext/projects/pasteur/src/pasteur/synth/base.py#76\76]8;;\
                             <np.random> 0.71178 <random> 0.14817                                                  